# Revising manuscript. 
## Data preparing according to reviewer's comments.

In [123]:
import pandas as pd

In [124]:
cre = pd.read_csv('ephys_data_cre_EI_180116.csv') # use this data for using specimen id
# cre = pd.read_csv('ephys_data_cre.csv')
# as review 2 minor commemts # 11 . technical values removed.
cre = cre.drop(['reporter_status', 'height', 'input_resistance_mohm', 'vm_for_sag', 
                'slow_trough_t_short_square', 'slow_trough_v_short_square', 
                'slow_trough_t_long_square', 'slow_trough_v_long_square',
               'slow_trough_t_ramp', 'slow_trough_v_ramp'], axis=1)
cre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 40 columns):
specimen_id                               969 non-null int64
adaptation                                816 non-null float64
avg_isi                                   889 non-null float64
f_i_curve_slope                           969 non-null float64
fast_trough_t_long_square                 969 non-null float64
fast_trough_t_ramp                        934 non-null float64
fast_trough_t_short_square                969 non-null float64
fast_trough_v_long_square                 969 non-null float64
fast_trough_v_ramp                        934 non-null float64
fast_trough_v_short_square                969 non-null float64
latency                                   969 non-null float64
peak_t_long_square                        969 non-null float64
peak_t_ramp                               934 non-null float64
peak_t_short_square                       969 non-null float64
peak_v_long_squ

adaptation                                816 non-null float64


In [125]:
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.core.cell_types_cache import ReporterStatus as RS

In [126]:
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')
cells = ctc.get_cells()
print("All cells: %d" % len(cells)) # load data, for including layer metadata.

All cells: 2333


In [127]:
cell_index = { c['id']: c for c in cells }

In [128]:
ephys_features_new = ctc.get_ephys_features()

In [129]:
ephys_features_new_df = pd.DataFrame(ephys_features_new)

In [130]:
tau = [cell_index[cid]['ephys_features'][0]['tau'] for cid in ephys_features_new_df['specimen_id']]

In [131]:
ephys_features_new_df['tau'] = pd.Series(tau, index=ephys_features_new_df.index)
ephys_features_new_df.head()

,adaptation,avg_isi,electrode_0_pa,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,...,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest,tau
0,NaN,134.700000,22.697498,8.335459e-02,1.187680,13.295200,1.025916,-56.375004,-57.385420,-57.431251,...,1.134780,-56.593754,-57.739586,-74.143753,3.029695,3.061646,2.969821,-80.468750,-73.553391,20.495034
1,NaN,NaN,-24.887498,-3.913630e-19,1.099840,20.650105,1.025460,-54.000000,-54.828129,-54.656254,...,1.160940,-55.406254,-55.242191,-73.500000,2.441895,2.245653,2.231575,-84.406258,-73.056595,25.131172
2,0.009770,39.044800,-46.765002,5.267857e-01,1.157840,2.551310,1.025387,-59.500000,-58.234378,-59.940975,...,1.089851,-60.062500,-58.570314,-61.371531,2.023762,2.162878,2.006406,-93.375008,-60.277321,18.314883
3,-0.007898,117.816429,5.996250,1.542553e-01,1.989165,9.572025,1.028733,-47.531250,-50.359375,-65.500000,...,1.423229,-49.406254,-52.718752,-75.273443,3.105931,3.491663,1.733896,-87.656250,-75.205559,7.616164
4,0.022842,68.321429,14.910000,1.714041e-01,1.081980,2.462880,1.025620,-48.437504,-46.520837,-51.406253,...,1.479690,-53.000004,-54.645837,-64.250003,3.285760,3.363504,4.234701,-81.625008,-63.474991,34.829629


In [134]:
cell_index[489923501]['structure']['name']

u'Primary visual area, layer 2/3'

In [132]:
layer_full = [cell_index[cid]['structure']['name'] for cid in ephys_features_new_df['specimen_id']]

In [135]:
ephys_features_new_df['layer_full'] = pd.Series(layer_full, index=ephys_features_new_df.index)
ephys_features_new_df.head()

,adaptation,avg_isi,electrode_0_pa,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,...,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest,tau,layer_full
0,NaN,134.700000,22.697498,8.335459e-02,1.187680,13.295200,1.025916,-56.375004,-57.385420,-57.431251,...,-56.593754,-57.739586,-74.143753,3.029695,3.061646,2.969821,-80.468750,-73.553391,20.495034,middle temporal gyrus
1,NaN,NaN,-24.887498,-3.913630e-19,1.099840,20.650105,1.025460,-54.000000,-54.828129,-54.656254,...,-55.406254,-55.242191,-73.500000,2.441895,2.245653,2.231575,-84.406258,-73.056595,25.131172,middle frontal gyrus
2,0.009770,39.044800,-46.765002,5.267857e-01,1.157840,2.551310,1.025387,-59.500000,-58.234378,-59.940975,...,-60.062500,-58.570314,-61.371531,2.023762,2.162878,2.006406,-93.375008,-60.277321,18.314883,"Primary visual area, layer 5"
3,-0.007898,117.816429,5.996250,1.542553e-01,1.989165,9.572025,1.028733,-47.531250,-50.359375,-65.500000,...,-49.406254,-52.718752,-75.273443,3.105931,3.491663,1.733896,-87.656250,-75.205559,7.616164,"Primary visual area, layer 2/3"
4,0.022842,68.321429,14.910000,1.714041e-01,1.081980,2.462880,1.025620,-48.437504,-46.520837,-51.406253,...,-53.000004,-54.645837,-64.250003,3.285760,3.363504,4.234701,-81.625008,-63.474991,34.829629,"Primary visual area, layer 4"


In [136]:
layer_acronym = [cell_index[cid]['structure']['acronym'] for cid in ephys_features_new_df['specimen_id']]

In [137]:
ephys_features_new_df['acronym'] = pd.Series(layer_acronym, index=ephys_features_new_df.index)
ephys_features_new_df.head()

,adaptation,avg_isi,electrode_0_pa,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,...,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vm_for_sag,vrest,tau,layer_full,acronym
0,NaN,134.700000,22.697498,8.335459e-02,1.187680,13.295200,1.025916,-56.375004,-57.385420,-57.431251,...,-57.739586,-74.143753,3.029695,3.061646,2.969821,-80.468750,-73.553391,20.495034,middle temporal gyrus,MTG
1,NaN,NaN,-24.887498,-3.913630e-19,1.099840,20.650105,1.025460,-54.000000,-54.828129,-54.656254,...,-55.242191,-73.500000,2.441895,2.245653,2.231575,-84.406258,-73.056595,25.131172,middle frontal gyrus,MFG
2,0.009770,39.044800,-46.765002,5.267857e-01,1.157840,2.551310,1.025387,-59.500000,-58.234378,-59.940975,...,-58.570314,-61.371531,2.023762,2.162878,2.006406,-93.375008,-60.277321,18.314883,"Primary visual area, layer 5",VISp5
3,-0.007898,117.816429,5.996250,1.542553e-01,1.989165,9.572025,1.028733,-47.531250,-50.359375,-65.500000,...,-52.718752,-75.273443,3.105931,3.491663,1.733896,-87.656250,-75.205559,7.616164,"Primary visual area, layer 2/3",VISp2/3
4,0.022842,68.321429,14.910000,1.714041e-01,1.081980,2.462880,1.025620,-48.437504,-46.520837,-51.406253,...,-54.645837,-64.250003,3.285760,3.363504,4.234701,-81.625008,-63.474991,34.829629,"Primary visual area, layer 4",VISp4


In [138]:
selected_df = ephys_features_new_df[['specimen_id', 'tau', 'layer_full', 'acronym']]
selected_df.head()

,specimen_id,tau,layer_full,acronym
0,529878215,20.495034,middle temporal gyrus,MTG
1,548459652,25.131172,middle frontal gyrus,MFG
2,579978640,18.314883,"Primary visual area, layer 5",VISp5
3,439024551,7.616164,"Primary visual area, layer 2/3",VISp2/3
4,515188639,34.829629,"Primary visual area, layer 4",VISp4


In [139]:
cre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 40 columns):
specimen_id                               969 non-null int64
adaptation                                816 non-null float64
avg_isi                                   889 non-null float64
f_i_curve_slope                           969 non-null float64
fast_trough_t_long_square                 969 non-null float64
fast_trough_t_ramp                        934 non-null float64
fast_trough_t_short_square                969 non-null float64
fast_trough_v_long_square                 969 non-null float64
fast_trough_v_ramp                        934 non-null float64
fast_trough_v_short_square                969 non-null float64
latency                                   969 non-null float64
peak_t_long_square                        969 non-null float64
peak_t_ramp                               934 non-null float64
peak_t_short_square                       969 non-null float64
peak_v_long_squ

In [141]:
new_cre = pd.merge(selected_df, cre, on='specimen_id', how = 'inner' )

In [142]:
new_cre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 965 entries, 0 to 964
Data columns (total 43 columns):
specimen_id                               965 non-null int64
tau                                       965 non-null float64
layer_full                                965 non-null object
acronym                                   965 non-null object
adaptation                                812 non-null float64
avg_isi                                   885 non-null float64
f_i_curve_slope                           965 non-null float64
fast_trough_t_long_square                 965 non-null float64
fast_trough_t_ramp                        930 non-null float64
fast_trough_t_short_square                965 non-null float64
fast_trough_v_long_square                 965 non-null float64
fast_trough_v_ramp                        930 non-null float64
fast_trough_v_short_square                965 non-null float64
latency                                   965 non-null float64
peak_t_long_squar

In [143]:
cre_dropna = new_cre.dropna(axis=0, how='any')

In [146]:
a = new_cre['transgenic_line'].value_counts()
b= cre_dropna['transgenic_line'].value_counts()
a - b

Chat-IRES-Cre-neo    11
Chrna2-Cre_OE25       1
Ctgf-T2A-dgCre        0
Cux2-CreERT2         13
Gad2-IRES-Cre         4
Htr3a-Cre_NO152      16
Ndnf-IRES2-dgCre      1
Nkx2-1-CreERT2        1
Nr5a1-Cre             9
Ntsr1-Cre_GN220      15
Pvalb-IRES-Cre       38
Rbp4-Cre_KL100       13
Rorb-IRES2-Cre       14
Scnn1a-Tg2-Cre        3
Scnn1a-Tg3-Cre        7
Slc17a6-IRES-Cre      1
Sst-IRES-Cre         20
Vip-IRES-Cre          1
Name: transgenic_line, dtype: int64

In [147]:
(a-b)/a

Chat-IRES-Cre-neo    0.550000
Chrna2-Cre_OE25      0.032258
Ctgf-T2A-dgCre       0.000000
Cux2-CreERT2         0.178082
Gad2-IRES-Cre        0.363636
Htr3a-Cre_NO152      0.181818
Ndnf-IRES2-dgCre     0.066667
Nkx2-1-CreERT2       0.043478
Nr5a1-Cre            0.132353
Ntsr1-Cre_GN220      0.312500
Pvalb-IRES-Cre       0.267606
Rbp4-Cre_KL100       0.180556
Rorb-IRES2-Cre       0.118644
Scnn1a-Tg2-Cre       0.083333
Scnn1a-Tg3-Cre       0.116667
Slc17a6-IRES-Cre     0.333333
Sst-IRES-Cre         0.180180
Vip-IRES-Cre         0.050000
Name: transgenic_line, dtype: float64

In [158]:
new_cre.loc[new_cre['transgenic_line'] == 'Ndnf-IRES2-dgCre', ['binary_neuron']] = 'Inhibitory'
new_cre.loc[new_cre['transgenic_line'] == 'Ndnf-IRES2-dgCre']

,specimen_id,tau,layer_full,acronym,adaptation,avg_isi,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,...,trough_t_short_square,trough_v_long_square,trough_v_ramp,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vrest,transgenic_line,binary_neuron
141,530235049,8.629247,"Primary visual area, layer 2/3",VISp2/3,0.002679,27.175000,0.335919,1.11192,7.368600,1.024668,...,1.052460,-53.625004,-53.000001,-70.387503,2.502756,2.628995,2.568368,-69.332764,Ndnf-IRES2-dgCre,Inhibitory
221,526531616,9.050988,"Primary visual area, layer 1",VISp1,0.006942,28.112353,0.303640,1.53710,6.348080,1.026160,...,1.343004,-54.500004,-52.010418,-72.425003,2.980973,2.859881,2.546601,-72.557991,Ndnf-IRES2-dgCre,Inhibitory
282,526939383,10.630557,"Primary visual area, layer 1",VISp1,0.002607,31.416129,0.340848,1.09790,6.254480,1.025248,...,1.056428,-52.343754,-52.479169,-72.431255,2.717109,2.526602,2.358213,-71.806557,Ndnf-IRES2-dgCre,Inhibitory
384,569623233,8.774123,"Primary visual area, layer 2/3",VISp2/3,-0.018631,35.221481,0.459787,1.91420,7.777170,1.025636,...,1.203100,-47.968754,-51.359377,-73.037505,2.542889,2.900498,2.491942,-72.636589,Ndnf-IRES2-dgCre,Inhibitory
474,579981789,16.705617,"Primary visual area, layer 6b",VISp6b,0.018214,75.793333,0.210757,1.11860,4.108287,1.025616,...,1.667028,-56.312500,-57.260418,-73.056253,3.630774,3.674494,3.922287,-72.164597,Ndnf-IRES2-dgCre,Inhibitory
477,527095729,8.441577,"Primary visual area, layer 1",VISp1,-0.011970,25.275897,0.440534,1.04206,11.520100,1.027056,...,1.185696,-51.500004,-50.645836,-76.043753,2.877833,3.136740,2.715563,-74.506088,Ndnf-IRES2-dgCre,Inhibitory
500,565778331,15.529607,"Primary visual area, layer 5",VISp5,0.050492,104.200000,0.136731,1.16406,5.423500,1.024932,...,1.092216,-54.093754,-46.500003,-67.843753,3.286742,3.239252,3.029947,-66.556534,Ndnf-IRES2-dgCre,Inhibitory
602,527579814,4.763883,"Primary visual area, layer 1",VISp1,0.005303,52.404706,0.328125,1.83288,8.159487,1.027337,...,1.374460,-52.031254,-54.750001,-76.656253,3.979260,4.137271,2.575332,-76.635284,Ndnf-IRES2-dgCre,Inhibitory
613,564346637,25.978014,"Primary visual area, layer 5",VISp5,NaN,20.160000,0.665800,1.06240,6.785400,1.024127,...,1.178143,-60.968754,-52.593754,-63.812503,1.480181,1.053327,1.613715,-62.698147,Ndnf-IRES2-dgCre,Inhibitory
709,579611597,8.572176,"Primary visual area, layer 2/3",VISp2/3,-0.025917,79.514545,0.257085,1.85968,9.415487,1.027660,...,1.229840,-54.000000,-53.052087,-76.406258,3.787364,3.863806,2.875604,-75.740387,Ndnf-IRES2-dgCre,Inhibitory


In [157]:
new_cre[['transgenic_line', 'acronym', 'binary_neuron']]

,transgenic_line,acronym,binary_neuron
0,Scnn1a-Tg3-Cre,VISp4,Excitatory
1,Pvalb-IRES-Cre,VISp5,Inhibitory
2,Ntsr1-Cre_GN220,VISp6a,Excitatory
3,Chat-IRES-Cre-neo,VISp4,Inhibitory
4,Pvalb-IRES-Cre,VISp2/3,Inhibitory
5,Nr5a1-Cre,VISp4,Excitatory
6,Gad2-IRES-Cre,VISp5,Inhibitory
7,Sst-IRES-Cre,VISpor5,Inhibitory
8,Sst-IRES-Cre,VISp5,Inhibitory
9,Gad2-IRES-Cre,VISp5,Inhibitory


In [167]:
new_cre[new_cre['transgenic_line'] == 'Nr5a1-Cre']['layer_full'].value_counts()
# TODO_ how to handle layer information

Primary visual area, layer 4            42
Primary visual area, layer 2/3          14
posteromedial visual area, layer 4       4
Primary visual area, layer 5             2
Primary visual area, layer 6a            2
posteromedial visual area, layer 2/3     2
Anteromedial visual area, layer 2/3      1
Anterolateral visual area, layer 4       1
Name: layer_full, dtype: int64

In [166]:
cre['peak_t_short_square']= (cre['peak_t_short_square']-1.02)*1000
cre['peak_t_short_square']
# make time unit as 3ms, short square pulse stimuli applied 3ms, so the start of stimuli set to zero.

0      3.300000
1      3.480000
2      3.475000
3      3.508333
4      3.560000
5      3.597000
6      3.541000
7      3.311000
8      3.390000
9      3.570000
10     3.478000
11     3.030625
12     3.508333
13     3.841667
14     3.140000
15     3.462000
16     3.427500
17     3.160000
18     3.318750
19     3.580000
20     3.336000
21     3.432000
22     3.256667
23     3.460000
24     3.657000
25     3.525000
26     3.320000
27     3.673333
28     3.672000
29     3.344000
         ...   
939    3.430000
940    3.392000
941    3.365000
942    3.735000
943    3.420000
944    3.373000
945    3.436250
946    3.405000
947    3.410000
948    3.280000
949    3.515000
950    3.385000
951    3.342500
952    3.215000
953    3.227000
954    3.710000
955    3.448750
956    3.315000
957    3.552000
958    3.524000
959    3.343750
960    3.292000
961    3.447500
962    3.303000
963    3.493750
964    3.413750
965    3.576000
966    3.410000
967    3.445000
968    3.592000
Name: peak_t_short_squar

In [167]:
cre['threshold_t_short_square'] = (cre['threshold_t_short_square']-1.02)*1000
cre['threshold_t_short_square']

0      2.913333
1      3.159000
2      3.001000
3      3.130000
4      3.120000
5      2.979000
6      3.197000
7      3.032000
8      3.010000
9      3.142000
10     2.943000
11     2.731875
12     3.150000
13     3.266667
14     2.917000
15     2.979000
16     3.043750
17     2.746667
18     3.002500
19     3.103000
20     2.946000
21     3.123000
22     2.856667
23     3.104000
24     3.137000
25     3.155000
26     2.970000
27     3.191667
28     3.168000
29     2.985000
         ...   
939    3.055000
940    3.055000
941    2.948333
942    3.289000
943    3.092500
944    3.033000
945    3.061250
946    3.046000
947    3.025000
948    2.970000
949    3.114000
950    3.055000
951    3.040000
952    2.895000
953    2.994000
954    3.190000
955    3.090000
956    3.053000
957    3.120000
958    3.144000
959    2.983750
960    3.008000
961    3.077500
962    3.033000
963    3.192500
964    3.025000
965    3.132000
966    3.095000
967    3.065000
968    3.164000
Name: threshold_t_short_

In [168]:
cre['fast_trough_t_short_square'] = (cre['fast_trough_t_short_square']-1.02)*1000
cre['fast_trough_t_short_square']

0      4.873333
1      4.593000
2      7.302000
3      4.665833
4      5.426667
5      5.615000
6      4.461000
7      4.103000
8      4.870000
9      5.506000
10     5.854000
11     3.790625
12     4.195833
13     5.882500
14     3.625000
15     5.411000
16     4.420000
17     4.846667
18     4.180000
19     5.553000
20     5.011000
21     4.358000
22     4.471667
23     5.428000
24     7.009000
25     4.895000
26     4.625000
27     6.062500
28     5.740000
29     4.485000
         ...   
939    4.815000
940    4.841000
941    5.006667
942    6.266000
943    5.032500
944    4.921000
945    9.810000
946    6.005000
947    5.352500
948    4.520000
949    7.092000
950    4.130000
951    8.495000
952    4.122500
953    3.877000
954    6.587857
955    5.132500
956    3.969000
957    5.192000
958    5.352000
959    5.056250
960    4.348000
961    9.131250
962    3.975000
963    4.292500
964    5.281250
965    5.360000
966    4.603750
967    4.965000
968    5.576000
Name: fast_trough_t_shor

In [169]:
cre['trough_t_short_square'] = (cre['trough_t_short_square']-1.02)*1000
cre['trough_t_short_square']

0      324.913333
1      213.473000
2      671.744000
3      475.571667
4      110.500000
5      453.574000
6      527.561000
7      436.287000
8      489.350000
9      231.151000
10     557.570000
11     343.485625
12     377.621667
13      34.124167
14     673.986000
15     391.247000
16     291.541250
17     811.826667
18     230.333750
19     426.244000
20     592.528000
21     395.886000
22     810.231667
23     585.676000
24     505.379000
25     518.510000
26     835.710000
27     527.079167
28      97.364000
29     720.821000
          ...    
939    669.315000
940    510.804000
941    689.963333
942    338.333000
943    446.345000
944    510.155000
945     27.485000
946    129.956000
947     66.213333
948    585.140000
949    589.005000
950    227.725000
951    179.678750
952      4.257500
953    612.955000
954    618.815000
955    619.258750
956    561.903000
957    584.992000
958    124.844000
959    682.153750
960    458.228000
961    512.405000
962     68.214000
963     26

In [170]:
cre['threshold_t_long_square'] = (cre['threshold_t_long_square']-1.02)*1000
cre['peak_t_long_square'] = (cre['peak_t_long_square']-1.02)*1000
cre['fast_trough_t_long_square'] = (cre['fast_trough_t_long_square']-1.02)*1000
cre['trough_t_long_square'] = (cre['trough_t_long_square']-1.02)*1000
# make time unit as ms, long square pulse stimuli applied for 1 s from 100 ms of sweep
# the start of stimuli set to zero.

In [171]:
cre['threshold_t_ramp'] = cre['threshold_t_ramp'] - 1.02
cre['peak_t_ramp'] = cre['peak_t_ramp'] -1.02
cre['fast_trough_t_ramp'] = cre['fast_trough_t_ramp'] - 1.02
cre['trough_t_ramp'] = cre['trough_t_ramp'] - 1.02
# ramp stimuli applied from 1.0 s of sweep
# the start of stimuli set to zero.

In [172]:
cre['height_short'] = cre['peak_v_short_square'] -cre['fast_trough_v_short_square']
cre['height_long'] = cre['peak_v_long_square'] - cre['fast_trough_v_long_square']
cre['height_ramp'] = cre['peak_v_ramp'] - cre['fast_trough_v_ramp']

In [173]:
(cre['height_long'] - cre['height_short']).mean

<bound method Series.mean of 0       0.187496
1     -22.099999
2      -2.700003
3      -8.223962
4      -9.635417
5      -6.293752
6      -6.718746
7     -13.662500
8      -4.562498
9      -8.037503
10     -4.874999
11     -3.972647
12     -0.749999
13    -11.859377
14    -13.056254
15     -4.850003
16     -2.234374
17     -2.489583
18     -3.273437
19     -7.656248
20    -10.331248
21     -0.293748
22     -1.500002
23     -8.206251
24    -10.037502
25     -3.007812
26     -2.718750
27     -6.911457
28     -9.443750
29     -1.706247
         ...    
939    -6.218749
940    -6.118752
941     6.052085
942    -7.125002
943   -27.765625
944    -3.981248
945    -4.148438
946    -5.131252
947    -8.656249
948    -2.007816
949    -9.474995
950    -4.367189
951   -11.320311
952   -13.906254
953    -4.537498
954    -7.058038
955    -9.882810
956   -15.675001
957   -10.212505
958    -3.600000
959    -0.414067
960   -11.150001
961   -13.718753
962    -9.350000
963    -7.437502
964   -10.773438
96

In [174]:
(cre['height_long'] - cre['height_ramp']).mean

<bound method Series.mean of 0      -0.833333
1     -20.000001
2      -7.718753
3       3.843746
4      -1.875001
5     -10.916670
6       2.427087
7      -0.020831
8      -3.458332
9     -16.208337
10     -1.562497
11           NaN
12      2.197920
13     -9.645837
14     14.249996
15     -3.635420
16     -4.249997
17     -2.062499
18     -0.552080
19      3.520835
20      7.552085
21      2.875002
22     -0.812500
23     -4.624996
24     -5.104167
25     -0.937500
26     -0.187496
27           NaN
28     -3.979164
29      1.250004
         ...    
939    -1.197917
940     3.781252
941     3.552086
942    -4.250003
943   -15.895835
944    -6.458330
945   -10.729166
946    -2.364583
947    -3.291666
948     0.291660
949    -4.218744
950     4.999998
951    -7.124999
952   -15.687505
953     7.833336
954   -13.145835
955    -0.604164
956     8.656252
957    -4.197919
958    -2.447918
959     0.156244
960     4.250002
961   -10.531254
962    24.125000
963    13.953124
964   -11.302085
96

In [175]:
(cre['height_ramp'] - cre['height_short']).mean

<bound method Series.mean of 0       1.020830
1      -2.099998
2       5.018749
3     -12.067707
4      -7.760415
5       4.622918
6      -9.145833
7     -13.641668
8      -1.104166
9       8.170833
10     -3.312502
11           NaN
12     -2.947919
13     -2.213540
14    -27.306250
15     -1.214583
16      2.015624
17     -0.427085
18     -2.721357
19    -11.177083
20    -17.883333
21     -3.168750
22     -0.687502
23     -3.581255
24     -4.933336
25     -2.070312
26     -2.531254
27           NaN
28     -5.464586
29     -2.956251
         ...    
939    -5.020832
940    -9.900003
941     2.499999
942    -2.874999
943   -11.869790
944     2.477082
945     6.580729
946    -2.766668
947    -5.364583
948    -2.299477
949    -5.256251
950    -9.367187
951    -4.195312
952     1.781251
953   -12.370834
954     6.087796
955    -9.278646
956   -24.331252
957    -6.014586
958    -1.152082
959    -0.570312
960   -15.400003
961    -3.187499
962   -33.475000
963   -21.390626
964     0.528647
96

In [176]:
cre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 43 columns):
adaptation                                816 non-null float64
avg_isi                                   889 non-null float64
f_i_curve_slope                           969 non-null float64
fast_trough_t_long_square                 969 non-null float64
fast_trough_t_ramp                        934 non-null float64
fast_trough_t_short_square                969 non-null float64
fast_trough_v_long_square                 969 non-null float64
fast_trough_v_ramp                        934 non-null float64
fast_trough_v_short_square                969 non-null float64
latency                                   969 non-null float64
peak_t_long_square                        969 non-null float64
peak_t_ramp                               934 non-null float64
peak_t_short_square                       969 non-null float64
peak_v_long_square                        969 non-null float64
peak_v_ramp  

,adaptation,avg_isi,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,latency,...,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vrest,transgenic_line,firing_rate,binary_neuron,height_short,height_long,height_ramp
70,-0.016836,51.317500,0.253219,510.66,3.336407,5.040000,-53.656254,-53.114587,-56.575002,0.14006,...,3.277697,3.293132,2.619381,-67.046272,Ndnf-IRES2-dgCre,19.486530,Inhibitory,86.106253,80.156256,79.562504
99,0.008434,36.099259,0.480000,27.64,6.622927,7.312000,-47.437504,-46.395836,-56.981253,0.01428,...,2.780688,3.089430,2.601046,-72.885735,Ndnf-IRES2-dgCre,27.701399,Inhibitory,84.543754,72.437504,69.708337
258,0.005303,52.404706,0.328125,812.88,7.139487,7.336667,-49.937500,-51.989586,-60.036462,0.08436,...,3.979260,4.137271,2.575332,-76.635284,Ndnf-IRES2-dgCre,19.082256,Inhibitory,89.848962,76.843752,80.812504
264,0.002679,27.175000,0.335919,91.92,6.348600,4.668000,-53.500004,-52.239586,-55.675002,0.00868,...,2.502756,2.628995,2.568368,-69.332764,Ndnf-IRES2-dgCre,36.798528,Inhibitory,88.375003,80.906256,83.468754
356,NaN,20.160000,0.665800,42.40,5.765400,4.126667,-60.875004,-52.406250,-61.963545,0.01342,...,1.480181,1.053327,1.613715,-62.698147,Ndnf-IRES2-dgCre,49.603175,Inhibitory,91.453130,87.656256,63.343751
456,0.006942,28.112353,0.303640,517.10,5.328080,6.160000,-53.437500,-50.052087,-55.975002,0.02890,...,2.980973,2.859881,2.546601,-72.557991,Ndnf-IRES2-dgCre,35.571551,Inhibitory,89.081253,79.062502,77.041672
460,-0.025917,79.514545,0.257085,839.68,8.395487,7.660000,-52.406250,-49.739585,-58.562504,0.10944,...,3.787364,3.863806,2.875604,-75.740387,Ndnf-IRES2-dgCre,12.576315,Inhibitory,87.531256,66.281251,67.614587
525,-0.000414,31.932143,0.391826,947.66,5.427647,5.716667,-50.656250,-51.541669,-55.088544,0.09760,...,3.023153,3.061118,2.408996,-65.245323,Ndnf-IRES2-dgCre,31.316408,Inhibitory,78.812503,68.656252,69.645837
574,0.002607,31.416129,0.340848,77.90,5.234480,5.248000,-48.968750,-49.156253,-51.343752,0.01168,...,2.717109,2.526602,2.358213,-71.806557,Ndnf-IRES2-dgCre,31.830783,Inhibitory,78.300003,70.468750,73.104169
704,-0.007651,37.511200,0.312106,453.92,7.494600,8.415000,-52.656254,-49.458336,-60.109377,0.03728,...,3.007957,3.439152,2.371974,-75.434464,Ndnf-IRES2-dgCre,26.658705,Inhibitory,83.664066,71.656256,71.427088


In [178]:
cre.to_csv('cre_revised.csv', index = False)